<a href="https://colab.research.google.com/github/andres-stack/Stock-Price-Perdiction/blob/main/svmStock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
import yfinance as yf
import datetime
from finta import TA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
numDays = 14000
dataInterval = '1d'
symbol = 'TSLA'
INDICATORS = INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']


start = (datetime.date.today() - datetime.timedelta(numDays))
end = datetime.datetime.today()

data = yf.download(symbol, start=start, end=end, interval=dataInterval)
data.rename(columns={"Close": 'close', "High": 'high', "Low": 'low', 'Volume': 'volume', 'Open': 'open'}, inplace=True)

print(data)

In [16]:
#Feature Engineering Technical Indicators
for indicator in INDICATORS:
  indicatorValues = eval('TA.' + indicator + '(data)')
  if not isinstance(indicatorValues, pd.DataFrame):
    indicatorValues = indicatorValues.to_frame()
  data = data.merge(indicatorValues, left_index=True, right_index=True)
del (data['open'])
del (data['high'])
del (data['low'])
del (data['volume'])
del (data['Adj Close'])

In [ ]:
#finding the truth values
def getprediction(data, window):
    
    prediction = (data.shift(-window)['close'] >= data['close'])
    prediction = prediction.iloc[:-window]
    data['pred'] = prediction.astype(int)
    
    return data

data = getprediction(data, window=15)
del (data['close'])
data = data.dropna() 
data.tail()

In [18]:
#implementing SVM
X = data.drop('pred', axis=1)
y = data['pred']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [20]:
#feature Scaling
from sklearn import preprocessing
import numpy as np
scaler = preprocessing.StandardScaler().fit(X_train)
scaler = preprocessing.StandardScaler().fit(X_test)
X_testScaled = scaler.transform(X_test)
X_scaled = scaler.transform(X_train)

In [10]:
svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_scaled, y_train)

SVC()

In [21]:
y_pred = svclassifier.predict(X_testScaled)

In [ ]:
from sklearn import metrics
print("Gaussian Kernel Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
#getting accuracy for Different plynomial terms
i = 2
while i < 20:
  polyclassifier = SVC(kernel = 'poly', degree = i)
  polyclassifier.fit(X_scaled, y_train)
  prediction = polyclassifier.predict(X_testScaled)
  print('polynomial degree '+str(i) +':',metrics.accuracy_score(y_test, prediction))
  i = i+1